### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#294B8E; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #294B8E">Разработка рекомендательной модели на основе данных о товарах</p>

<a id="1.2"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Содержание</div>

<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание данных</a></span></li>
    <li><span><a href="#Загрузка-библиотек" data-toc-modified-id="Загрузка-библиотек-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка библиотек</a></span></li>
    <li><span><a href="#EDA" data-toc-modified-id="EDA-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>EDA</a></span></li>
    <li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Предобработка данных</a></span></li>
    <li><span><a href="#Подготовим-выборки-для-обучения-моделей" data-toc-modified-id="Подготовим-выборки-для-обучения-моделей-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Подготовим выборки для обучения моделей</a></span></li>
    <li><span><a href="#Обучение-модели" data-toc-modified-id="Обучение-модели-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Обучение модели</a></span></li>
       <li><span><a href="#Проверка-на-тестовых-данных" data-toc-modified-id="Проверка-на-тестовых-данных-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Проверка на тестовых данных</a></span></li>
    <li><span><a href="#Вывод" data-toc-modified-id="Вывод-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Вывод</a></span></li>

<a id="Описание-данных"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Описание данных</div>

## marketing_dealer - список дилеров

## marketing_dealerprice - результат работы парсера площадок дилеров
<ul>
<li>product_key - уникальный номер позиции;</li>
<li>price - цена;</li>
<li>product_url - адрес страницы, откуда собраны данные;</li>
<li>product_name - заголовок продаваемого товара;</li>   
<li>date - дата получения информации;</li>
<li>dealer_id - идентификатор дилера (внешний ключ к marketing_dealer)</li>
</ul>

## marketing_product - список товаров, которые производит и распространяет заказчик
<ul>
<li>article - артикул товара;</li>
<li>ean_13 - код товара (см. EAN 13);</li>
<li>name - название товара;</li>
<li>cost - стоимость;</li>
<li>min_recommended_price - рекомендованная минимальная цена;</li>
<li>recommended_price - рекомендованная цена;</li>
<li>category_id - категория товара;</li>
<li>ozon_name - названиет товара на Озоне;</li>
<li>name_1c - название товара в 1C;</li>
<li>wb_name - название товара на Wildberries;</li>
<li>ozon_article - описание для Озон;</li>
<li>wb_article - артикул для Wildberries;</li>
<li>ym_article - артикул для Яндекс.Маркета;</li>
</ul>

## marketing_productdealerkey - таблица матчинга товаров заказчика и товаров дилеров
<ul>
<li>key - внешний ключ к marketing_dealerprice</li>
<li>product_id - внешний ключ к marketing_product</li>
<li>dealer_id - внешний ключ к marketing_dealer</li>
</ul>

<div class="alert alert-block alert-info" style="background-color:#FFFAF0;
                                                 color:black;
                                                 border-color:#DEB887;
                                                 border-radius:5px;
                                                 width:50%;
                                                 margin: auto;
                                                 text-align: left;">


Предлагать несколько товаров заказчика,
которые с наибольшей вероятностью соответствуют размечаемому товару дилера.

</div>

<a id="Загрузка-библиотек"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Загрузка библиотек</div>

In [1]:
import re
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
import warnings
warnings.filterwarnings("ignore")

<a id="EDA"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> EDA</div>

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">Изучим наши датафреймы.

In [2]:
marketing_dealer = pd.read_csv('marketing_dealer.csv', sep = ';')
marketing_dealer.head(5)

,id,name
0,1,Moi_vibor_WB
1,2,Akson
2,3,Bafus
3,5,Castorama
4,6,Cubatora


In [3]:
marketing_dealer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18 non-null     int64 
 1   name    18 non-null     object
dtypes: int64(1), object(1)
memory usage: 420.0+ bytes


In [4]:
marketing_dealerprice = pd.read_csv('marketing_dealerprice.csv', sep = ';')
marketing_dealerprice.head(5)

,id,product_key,price,product_url,product_name,date,dealer_id
0,2,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-11,2
1,3,546408,175.0,https://akson.ru//p/kontsentrat_prosept_multip...,"Концентрат Prosept Multipower для мытья полов,...",2023-07-11,2
2,4,546234,285.0,https://akson.ru//p/sredstvo_dlya_chistki_lyus...,Средство для чистки люстр Prosept Universal An...,2023-07-11,2
3,5,651258,362.0,https://akson.ru//p/udalitel_rzhavchiny_prosep...,"Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 0...",2023-07-11,2
4,6,546355,205.0,https://akson.ru//p/sredstvo_moyushchee_dlya_b...,Средство моющее для бани и сауны Prosept Multi...,2023-07-11,2


In [5]:
marketing_dealerprice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20416 entries, 0 to 20415
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            20416 non-null  int64  
 1   product_key   20416 non-null  object 
 2   price         20416 non-null  float64
 3   product_url   20182 non-null  object 
 4   product_name  20416 non-null  object 
 5   date          20416 non-null  object 
 6   dealer_id     20416 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 1.1+ MB


In [6]:
marketing_product = pd.read_csv('marketing_product.csv', sep = ';')
marketing_product.head(5)

,Unnamed: 0,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
0,0,245,008-1,4.680008e+12,Антисептик невымываемыйPROSEPT ULTRAконцентрат...,360.00,858.0,20.0,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,189522705.0,150033482.0,008-1,NaN
1,1,3,242-12,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,460.16,1075.0,NaN,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,NaN,NaN,NaN,NaN,NaN
2,2,443,0024-06 с,4.680008e+12,"Герметик акриловый цвет сосна, ф/п 600мл",307.00,644.0,25.0,Герметик акриловый для швов для деревянных дом...,"Герметик акриловый цвет сосна, ф/п 600мл",Герметик акриловый для швов для деревянных дом...,189522735.0,150126217.0,0024-06-с,NaN
3,3,147,305-2,4.610093e+12,Кондиционер для белья с ароматом королевского...,157.73,342.0,29.0,"Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose...",339377922.0,150032962.0,305-2,NaN
4,4,502,0024-7 б,NaN,"Герметик акриловой цвет Белый, 7 кг",NaN,NaN,NaN,NaN,NaN,NaN,189522867.0,150126216.0,0024-7-б,NaN


In [7]:
marketing_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         496 non-null    int64  
 1   id                 496 non-null    int64  
 2   article            496 non-null    object 
 3   ean_13             464 non-null    float64
 4   name               494 non-null    object 
 5   cost               491 non-null    float64
 6   recommended_price  491 non-null    float64
 7   category_id        447 non-null    float64
 8   ozon_name          458 non-null    object 
 9   name_1c            485 non-null    object 
 10  wb_name            455 non-null    object 
 11  ozon_article       365 non-null    float64
 12  wb_article         340 non-null    float64
 13  ym_article         337 non-null    object 
 14  wb_article_td      32 non-null     object 
dtypes: float64(6), int64(2), object(7)
memory usage: 58.3+ KB


In [8]:
marketing_productdealerkey = pd.read_csv('marketing_productdealerkey.csv', sep = ';')
marketing_productdealerkey.head(5)

,id,key,dealer_id,product_id
0,1,546227,2,12
1,2,651265,2,106
2,3,546257,2,200
3,4,546408,2,38
4,5,651258,2,403


In [9]:
marketing_productdealerkey.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1700 entries, 0 to 1699
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1700 non-null   int64 
 1   key         1700 non-null   object
 2   dealer_id   1700 non-null   int64 
 3   product_id  1700 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 53.3+ KB


<div class="alert alert-block alert-info" style="background-color:#FFFAF0;
                                                 color:black;
                                                 border-color:#DEB887;
                                                 border-radius:5px;
                                                 width:50%;
                                                 margin: auto;
                                                 text-align: left;">

<b>Вывод 
<ul>
<li>Обнаружены пропуски значений в датафрейме marketing_product.</li>
<li>В данных дубликаты удалены.</li>
</ul></div>

<a id="Предобработка-данных"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Предобработка данных</div>

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">Удалим пропуски в столбцах 'name', 'ozon_name', 'name_1c', 'wb_name'.

In [10]:
marketing_product.dropna(subset=['name', 'ozon_name', 'name_1c', 'wb_name'], inplace=True)

In [11]:
marketing_product.info()

<class 'pandas.core.frame.DataFrame'>
Index: 455 entries, 0 to 495
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         455 non-null    int64  
 1   id                 455 non-null    int64  
 2   article            455 non-null    object 
 3   ean_13             453 non-null    float64
 4   name               455 non-null    object 
 5   cost               455 non-null    float64
 6   recommended_price  455 non-null    float64
 7   category_id        427 non-null    float64
 8   ozon_name          455 non-null    object 
 9   name_1c            455 non-null    object 
 10  wb_name            455 non-null    object 
 11  ozon_article       360 non-null    float64
 12  wb_article         334 non-null    float64
 13  ym_article         331 non-null    object 
 14  wb_article_td      32 non-null     object 
dtypes: float64(6), int64(2), object(7)
memory usage: 56.9+ KB


<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">Создадим новые столбцы с объемами продуктов.

In [12]:
marketing_dealerprice['volume_product_name'] = marketing_dealerprice['product_name'].str[-5:].apply(lambda x: re.sub(r'[^\w\s]', '', x))
marketing_product['volume_name'] = marketing_product['name'].str[-5:].apply(lambda x: re.sub(r'[^\w\s]', '', x))
marketing_product['volume_wb_name'] = marketing_product['wb_name'].str[-5:].apply(lambda x: re.sub(r'[^\w\s]', '', x))
marketing_product['volume_ozon_name'] = marketing_product['ozon_name'].str[-5:].apply(lambda x: re.sub(r'[^\w\s]', '', x))
marketing_product['volume_name_1c'] = marketing_product['name_1c'].str[-5:].apply(lambda x: re.sub(r'[^\w\s]', '', x))

In [13]:
marketing_product.head(1)

,Unnamed: 0,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td,volume_name,volume_wb_name,volume_ozon_name,volume_name_1c
0,0,245,008-1,4.680008e+12,Антисептик невымываемыйPROSEPT ULTRAконцентрат...,360.0,858.0,20.0,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,189522705.0,150033482.0,008-1,NaN,1 л,1 л,1 л,1 л


In [14]:
#функция, которая проверяет, есть ли в столбце числа
def num_there(s):
    return any(i.isdigit() for i in s)

In [15]:
#столбцы, которые показывают True если в столбце есть число, если нет False
marketing_dealerprice['volume_product_name_2'] = marketing_dealerprice['volume_product_name'].apply(num_there)
marketing_product['volume_name_2'] = marketing_product['volume_name'].apply(num_there)
marketing_product['volume_wb_name_2'] = marketing_product['volume_wb_name'].apply(num_there)
marketing_product['volume_name_1c_2'] = marketing_product['volume_name_1c'].apply(num_there)
marketing_product['volume_ozon_name_2'] = marketing_product['volume_ozon_name'].apply(num_there)

In [16]:
marketing_product

,Unnamed: 0,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,...,ym_article,wb_article_td,volume_name,volume_wb_name,volume_ozon_name,volume_name_1c,volume_name_2,volume_wb_name_2,volume_name_1c_2,volume_ozon_name_2
0,0,245,008-1,4.680008e+12,Антисептик невымываемыйPROSEPT ULTRAконцентрат...,360.00,858.0,20.0,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,...,008-1,NaN,1 л,1 л,1 л,1 л,True,True,True,True
2,2,443,0024-06 с,4.680008e+12,"Герметик акриловый цвет сосна, ф/п 600мл",307.00,644.0,25.0,Герметик акриловый для швов для деревянных дом...,"Герметик акриловый цвет сосна, ф/п 600мл",...,0024-06-с,NaN,600мл,600мл,600мл,600мл,True,True,True,True
3,3,147,305-2,4.610093e+12,Кондиционер для белья с ароматом королевского...,157.73,342.0,29.0,"Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose...",...,305-2,NaN,2 л,2 л,2 л,2 л,True,True,True,True
5,5,220,051-6,4.610093e+12,Грунт БЕТОНКОНТАКТготовый состав / 6 кг,703.00,1339.0,26.0,"Грунт бетоноконтакт, для гладких поверхностей ...","Грунт бетоноконтакт, для гладких поверхностей ...",...,051-6,NaN,6 кг,6 кг,6 кг,6 кг,True,True,True,True
6,6,385,051-10,4.680008e+12,Грунт БЕТОНКОНТАКТготовый состав / 12 кг,1111.00,2505.0,26.0,"Грунт бетоноконтакт, для гладких поверхностей ...","Грунт бетоноконтакт, для гладких поверхностей ...",...,051-10,NaN,12 кг,12 кг,12 кг,12 кг,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,491,127,152-5,4.680008e+12,Средство для уборки помещений после пожара с д...,849.78,1985.0,40.0,Средство для уборки помещений после пожара с д...,Средство для уборки помещений после пожара с д...,...,152-5,NaN,5 л,5 л,5 л,5 л,True,True,True,True
492,492,160,289-1,4.680008e+12,"Жидкое моющее средство для стирки шерсти, шелк...",201.00,436.0,23.0,"Гель для стирки шерсти, шелка и деликатных тка...","Гель для стирки шерсти, шелка и деликатных тка...",...,289-1,289-10,1 л,1 л,1 л,1 л,True,True,True,True
493,493,74,192-05,4.680008e+12,Средство для чистки гриля и духовых шкафовCook...,108.60,254.0,49.0,Cредство для чистки гриля и духовок Cooky Gril...,Cредство для чистки гриля и духовок Cooky Gril...,...,192-05,192-050,05 л,0 мл,0 мл,0 мл,True,True,True,True
494,494,34,186-5,4.680008e+12,Средство для мытья полов с полимерным покрытие...,579.74,1354.0,40.0,Профессиональное средство для мытья полов с по...,Профессиональное средство для мытья полов с по...,...,186-5,NaN,5 л,5 л,5 л,5 л,True,True,True,True


In [17]:
marketing_dealerprice.loc[marketing_dealerprice['volume_product_name_2'] == False, 'volume_name'] = 'no volume'
marketing_product.loc[marketing_product['volume_name_2'] == False, 'volume_name'] = 'no volume'
marketing_product.loc[marketing_product['volume_wb_name_2'] == False, 'volume_wb_name'] = 'no volume'
marketing_product.loc[marketing_product['volume_ozon_name_2'] == False, 'volume_ozon_name'] = 'no volume'
marketing_product.loc[marketing_product['volume_name_1c_2'] == False, 'volume_name_1c'] = 'no volume'

In [18]:
marketing_product

,Unnamed: 0,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,...,ym_article,wb_article_td,volume_name,volume_wb_name,volume_ozon_name,volume_name_1c,volume_name_2,volume_wb_name_2,volume_name_1c_2,volume_ozon_name_2
0,0,245,008-1,4.680008e+12,Антисептик невымываемыйPROSEPT ULTRAконцентрат...,360.00,858.0,20.0,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,...,008-1,NaN,1 л,1 л,1 л,1 л,True,True,True,True
2,2,443,0024-06 с,4.680008e+12,"Герметик акриловый цвет сосна, ф/п 600мл",307.00,644.0,25.0,Герметик акриловый для швов для деревянных дом...,"Герметик акриловый цвет сосна, ф/п 600мл",...,0024-06-с,NaN,600мл,600мл,600мл,600мл,True,True,True,True
3,3,147,305-2,4.610093e+12,Кондиционер для белья с ароматом королевского...,157.73,342.0,29.0,"Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose...",...,305-2,NaN,2 л,2 л,2 л,2 л,True,True,True,True
5,5,220,051-6,4.610093e+12,Грунт БЕТОНКОНТАКТготовый состав / 6 кг,703.00,1339.0,26.0,"Грунт бетоноконтакт, для гладких поверхностей ...","Грунт бетоноконтакт, для гладких поверхностей ...",...,051-6,NaN,6 кг,6 кг,6 кг,6 кг,True,True,True,True
6,6,385,051-10,4.680008e+12,Грунт БЕТОНКОНТАКТготовый состав / 12 кг,1111.00,2505.0,26.0,"Грунт бетоноконтакт, для гладких поверхностей ...","Грунт бетоноконтакт, для гладких поверхностей ...",...,051-10,NaN,12 кг,12 кг,12 кг,12 кг,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,491,127,152-5,4.680008e+12,Средство для уборки помещений после пожара с д...,849.78,1985.0,40.0,Средство для уборки помещений после пожара с д...,Средство для уборки помещений после пожара с д...,...,152-5,NaN,5 л,5 л,5 л,5 л,True,True,True,True
492,492,160,289-1,4.680008e+12,"Жидкое моющее средство для стирки шерсти, шелк...",201.00,436.0,23.0,"Гель для стирки шерсти, шелка и деликатных тка...","Гель для стирки шерсти, шелка и деликатных тка...",...,289-1,289-10,1 л,1 л,1 л,1 л,True,True,True,True
493,493,74,192-05,4.680008e+12,Средство для чистки гриля и духовых шкафовCook...,108.60,254.0,49.0,Cредство для чистки гриля и духовок Cooky Gril...,Cредство для чистки гриля и духовок Cooky Gril...,...,192-05,192-050,05 л,0 мл,0 мл,0 мл,True,True,True,True
494,494,34,186-5,4.680008e+12,Средство для мытья полов с полимерным покрытие...,579.74,1354.0,40.0,Профессиональное средство для мытья полов с по...,Профессиональное средство для мытья полов с по...,...,186-5,NaN,5 л,5 л,5 л,5 л,True,True,True,True


In [19]:
marketing_dealerprice['len_product_name'] = marketing_dealerprice['product_name'].str.len()
marketing_product['len_wb_name'] = marketing_product['wb_name'].str.len()
marketing_product['len_name_1c'] = marketing_product['name_1c'].str.len()
marketing_product['len_ozon_name'] = marketing_product['ozon_name'].str.len()
marketing_product['len_name'] = marketing_product['name'].str.len()

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">Удаляем неинформативные столбцы.

In [20]:
marketing_product = marketing_product.drop(['ozon_article', 'wb_article',
                                            'ym_article', 'wb_article_td',
                                            'cost','article','ean_13',
                                            'category_id','volume_name_2',
                                            'volume_wb_name_2','volume_name_1c_2',
                                            'volume_ozon_name_2'], axis=1)

In [21]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [22]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/moonkerimka/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [23]:
#лемматизация
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    word_list = nltk.word_tokenize(text)
    lemm_text = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in word_list])
    return lemm_text

In [24]:
# напишем функцию, которая оставит в тексте только кириллические символы и пробелы
def clear_text(text):
    text = re.sub(r'[^a-zA-Z ][^а-яёА-ЯЁ]', ' ', text)
    return " ".join(text.split())

In [25]:
from tqdm.notebook import tqdm
tqdm.pandas()
marketing_dealerprice['lemm_product_name'] = marketing_dealerprice['product_name'].progress_apply(clear_text).progress_apply(lemmatize)
marketing_product['lemm_name'] = marketing_product['name'].progress_apply(clear_text).progress_apply(lemmatize)
marketing_product['lemm_wb_name'] = marketing_product['wb_name'].progress_apply(clear_text).progress_apply(lemmatize)
marketing_product['lemm_name_1c'] = marketing_product['name_1c'].progress_apply(clear_text).progress_apply(lemmatize)
marketing_product['lemm_ozon_name'] = marketing_product['ozon_name'].progress_apply(clear_text).progress_apply(lemmatize)

  0%|          | 0/20416 [00:00<?, ?it/s]

  0%|          | 0/20416 [00:00<?, ?it/s]

  0%|          | 0/455 [00:00<?, ?it/s]

  0%|          | 0/455 [00:00<?, ?it/s]

  0%|          | 0/455 [00:00<?, ?it/s]

  0%|          | 0/455 [00:00<?, ?it/s]

  0%|          | 0/455 [00:00<?, ?it/s]

  0%|          | 0/455 [00:00<?, ?it/s]

  0%|          | 0/455 [00:00<?, ?it/s]

  0%|          | 0/455 [00:00<?, ?it/s]

In [26]:
marketing_product.head(5)

,Unnamed: 0,id,name,recommended_price,ozon_name,name_1c,wb_name,volume_name,volume_wb_name,volume_ozon_name,volume_name_1c,len_wb_name,len_name_1c,len_ozon_name,len_name,lemm_name,lemm_wb_name,lemm_name_1c,lemm_ozon_name
0,0,245,Антисептик невымываемыйPROSEPT ULTRAконцентрат...,858.0,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,1 л,1 л,1 л,1 л,85,85,85,58,Антисепти невымываемы ROSEPT ULTRAконцентра л,Антисепти невымываемы дл ответственны конструк...,Антисепти невымываемы дл ответственны конструк...,Антисепти невымываемы дл ответственны конструк...
2,2,443,"Герметик акриловый цвет сосна, ф/п 600мл",644.0,Герметик акриловый для швов для деревянных дом...,"Герметик акриловый цвет сосна, ф/п 600мл",Герметик акриловый для швов для деревянных дом...,600мл,600мл,600мл,600мл,100,40,100,40,Гермети акриловы цве сосн 0мл,Гермети акриловы дл шво дл деревянны домо конс...,Гермети акриловы цве сосн 0мл,Гермети акриловы дл шво дл деревянны домо конс...
3,3,147,Кондиционер для белья с ароматом королевского...,342.0,"Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose...",2 л,2 л,2 л,2 л,69,69,69,83,Кондиционе дл бель аромато королевског Ирис ry...,Кондиционе дл бель `` Королевски Ири Prosept C...,Кондиционе дл бель `` Королевски Ири Prosept C...,Кондиционе дл бель `` Королевски Ири Prosept C...
5,5,220,Грунт БЕТОНКОНТАКТготовый состав / 6 кг,1339.0,"Грунт бетоноконтакт, для гладких поверхностей ...","Грунт бетоноконтакт, для гладких поверхностей ...","Грунт бетоноконтакт, для гладких поверхностей ...",6 кг,6 кг,6 кг,6 кг,124,124,124,39,Грун БЕТОНКОНТАКТготовы соста кг,Грун бетоноконтак дл гладки поверхносте пере н...,Грун бетоноконтак дл гладки поверхносте пере н...,Грун бетоноконтак дл гладки поверхносте пере н...
6,6,385,Грунт БЕТОНКОНТАКТготовый состав / 12 кг,2505.0,"Грунт бетоноконтакт, для гладких поверхностей ...","Грунт бетоноконтакт, для гладких поверхностей ...","Грунт бетоноконтакт, для гладких поверхностей ...",12 кг,12 кг,12 кг,12 кг,125,125,125,40,Грун БЕТОНКОНТАКТготовы соста кг,Грун бетоноконтак дл гладки поверхносте пере н...,Грун бетоноконтак дл гладки поверхносте пере н...,Грун бетоноконтак дл гладки поверхносте пере н...


<div class="alert alert-block alert-info" style="background-color:#FFFAF0;
                                                 color:black;
                                                 border-color:#DEB887;
                                                 border-radius:5px;
                                                 width:50%;
                                                 margin: auto;
                                                 text-align: left;">

<b>✍ Вывод</b>
        <ul>
<li>Текст леммитизирован и очищен.</li>
 
</ul>
</div>

In [27]:
#создадим счётчик
count_tf_idf = TfidfVectorizer()

#посчитаем TF-IDF для корпуса текстов функцией fit_transform() 
tf_idf_product_name = count_tf_idf.fit_transform(marketing_dealerprice['lemm_product_name'])
tf_idf_ozon_name = count_tf_idf.fit_transform(marketing_product['lemm_ozon_name'])
tf_idf_name_1c = count_tf_idf.fit_transform(marketing_product['lemm_name_1c'])
tf_idf_wb_name = count_tf_idf.fit_transform(marketing_product['lemm_wb_name'])
tf_idf_name = count_tf_idf.fit_transform(marketing_product['lemm_name'])